In [16]:
import pandas as pd
import random
from nltk.tokenize import TweetTokenizer
from nltk import classify
from nltk import NaiveBayesClassifier
from nltk import confusionmatrix
from nltk import FreqDist
from nltk.metrics import scores
from sklearn import metrics
import matplotlib.pyplot as plt

In [2]:
# Download annotated comments and annotations.
# If you're Tracy, Courtney, or Amandalynne, don't run this step
# because you already have the data! If you aren't us, you will
# probably need to do this step.
# It will take a while.
ANNOTATED_COMMENTS_URL = 'https://ndownloader.figshare.com/files/7038044'
ANNOTATIONS_URL = 'https://ndownloader.figshare.com/files/7383751'


def download_file(url, fname):
    urllib.request.urlretrieve(url, fname)

# not needed if files already exist in directory
#download_file(ANNOTATED_COMMENTS_URL, 'attack_annotated_comments.tsv')
#download_file(ANNOTATIONS_URL, 'attack_annotations.tsv')

In [3]:
# Read the data into a Pandas dataframe.
comments = pd.read_csv('../attack_annotated_comments.tsv', sep='\t', index_col=0)
annotations = pd.read_csv('../attack_annotations.tsv',  sep='\t')

# Label a comment as an attack if over half of annotators did so.
# We can tinker with this threshold later.
labels = annotations.groupby('rev_id')['attack'].mean() > 0.5

# Join labels and comments
comments['attack'] = labels

In [4]:
print('tokenizing...')

tknz = TweetTokenizer()
# Preprocess the data -- remove newlines, tabs, quotes
# Something to consider: remove Wikipedia style markup (::'s and =='s)
comments['comment'] = comments['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
comments['comment'] = comments['comment'].apply(lambda x: x.replace("TAB_TOKEN", " "))
comments['comment'] = comments['comment'].apply(lambda x: x.replace("`", " "))
comments['comment'] = comments['comment'].apply(lambda x: tknz.tokenize(x))

tokenizing...


In [5]:
# If you would like to use a small data set for debugging
#train_data = comments.iloc[0:100]
#test_data = comments.iloc[200:210]

# Grab the training data (seems to be 60%)
train_data = comments.loc[comments['split'] == 'train']
dev_data = comments.loc[comments['split'] == 'dev']
test_data = comments.loc[comments['split'] == 'test']

In [6]:
# The list of gold-standard labels for the data
train_labels = train_data["attack"].tolist()
dev_labels = dev_data["attack"].tolist()
test_labels = test_data["attack"].tolist()
# Put all the training data (comments) into a list
train_texts = train_data["comment"].tolist()
dev_texts = dev_data["comment"].tolist()
test_texts = test_data["comment"].tolist()

In [7]:
def randomly_sample(comments, labels):
    # grab all the attacks to see how many we need to match
    attack_indices = [i for i in range(len(comments)) if labels[i] == True]
    new_training = [comments[i] for i in attack_indices]
    new_labels = [labels[i] for i in attack_indices]

    # grab all the ones that are not attacks, shuffle them and select the same number of non-attacks
    non_attack_indices = [i for i in range(len(comments)) if labels[i] == False]
    random.shuffle(non_attack_indices)
    for i in range(len(attack_indices)):
        new_training.append(comments[i])
        new_labels.append(labels[i])

    # shuffle the training and labeling so that they're still matched 1:1
    shuf_indices = [i for i in range(len(new_training))]
    random.shuffle(shuf_indices)
    shuffled_training = [new_training[i] for i in shuf_indices]
    shuffled_labels = [new_labels[i] for i in shuf_indices]

    return shuffled_training, shuffled_labels

In [8]:
train_texts, train_labels = randomly_sample(train_texts, train_labels)
dev_texts, dev_labels = randomly_sample(dev_texts, dev_labels)
test_texts, test_labels = randomly_sample(test_texts, test_labels)

In [9]:
# Provide definitions for either bag of words or n-gram model
print('creating feature sets...')

# bag-of-words features
def word_feats(words):
    return dict([(word, True) for word in words])

# word frequency features
def word_freq(words):
    return dict(FreqDist(words))

# Create the training set and test set
train_features = [word_freq(x) for x in train_texts]
train_set = list(zip(train_features, train_labels))
test_features = [word_freq(x) for x in test_texts]
test_set = list(zip(test_features, test_labels))

creating feature sets...


In [10]:
print('classifying...')
# Classify and  retreive system labels
classifier = NaiveBayesClassifier.train(train_set)
sys_label = classifier.classify_many(test_features)

classifying...


In [26]:
# Extract Attack probabilities for AUROC measure
sys_prob = [classifier.prob_classify(test_features[j]).prob(True) for j in range(len(test_features))]
roc_auc = metrics.roc_auc_score(test_labels, sys_prob)
fpr, tpr, threshold = metrics.roc_curve(test_labels, sys_prob)

# Evaluation metrics
f1 = metrics.f1_score(test_labels, sys_label)
accuracy = classify.accuracy(classifier, test_set)
precision = metrics.precision_score(test_labels, sys_label)
recall = metrics.recall_score(test_labels, sys_label)
print('\nAccuracy: {}\nF1: {}\nAUC: {}\nPrec: {}\nRecall: {}'.format(accuracy, f1, roc_auc, precision, recall))


Accuracy: 0.7392960812772134
F1: 0.6949692209721927
AUC: 0.8409750996015937
Prec: 0.9635079458505003
Recall: 0.5434926958831341


In [12]:
# Print confusion matrix
cm = confusionmatrix.ConfusionMatrix(test_labels, sys_label)
print(cm.pretty_format(sort_by_count=True, show_percents=True, truncate=9))

      |             F |
      |      T      a |
      |      r      l |
      |      u      s |
      |      e      e |
------+---------------+
 True | <29.7%> 24.9% |
False |   1.1% <44.2%>|
------+---------------+
(row = reference; col = test)



In [13]:
# Plot the ROC
def plot_ROC(fpr, tpr, roc_auc):
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.savefig("NB_word_roc.png")
    
plot_ROC(fpr, tpr, roc_auc)

In [14]:
print(classifier.show_most_informative_features(5))

Most Informative Features
                    Fuck = 1                True : False  =    181.7 : 1.0
                    fuck = 1                True : False  =     99.1 : 1.0
                  faggot = 1                True : False  =     91.8 : 1.0
                 asshole = 1                True : False  =     82.6 : 1.0
                   bitch = 1                True : False  =     70.6 : 1.0
                 fucking = 1                True : False  =     61.7 : 1.0
                 fucking = 2                True : False  =     59.8 : 1.0
                    dick = 1                True : False  =     53.8 : 1.0
                 bastard = 1                True : False  =     52.1 : 1.0
                    cunt = 1                True : False  =     47.6 : 1.0
                   tests = 1               False : True   =     46.3 : 1.0
                    suck = 1                True : False  =     45.5 : 1.0
           experimenting = 1               False : True   =     43.1 : 1.0